This is my rather silly introduction to learning NLP. I think the show "Rick and Morty" is hilarious, so I trained a model to read every script from the show's 3 seasons and then had it generate new lines in the voice of specific characters. At the bottom of this file I have demonstrated 4 new lines for Rick because he is my favorite character, but the model is designed so that every character had a trained model, meaning a new line could be generated for any of the characters, in their voice. 

In [2]:
import nltk
#nltk.download('punkt')

from webscrape import getAndCleanWebScrapedContent,compileScript

all_the_scripts = compileScript()

In [3]:
#################################################
##### pre-procesing for sentiment analysis ######
#################################################

def preprocessingForSentimentAnalysis(script):
#strip punctuation and stopwords
    from string import punctuation
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words("english"))
    #this will split the script string on whitespace (by each word)
    split = script.split() 
        
    #loop through each word in script and if each word is 
    #not in stop_words, then add that word to the list of no_stopwords
    no_stopwords = []
    for word in split:
        if word.lower() not in stop_words:
            no_stopwords.append(word)
            
    #loop through each word in no_stopwords and if each item is 
    #not a punctuation, then add that item to the list of no_punctuation
    #this list contains only non-stopwords and does not include individual
    #items of punctuation that were before being interpreted as words
    no_punctuation_no_stopwords = []
    for word in no_stopwords:
        if word not in punctuation:
            no_punctuation_no_stopwords.append(word)
            
    #rejoin the list into one large string
    rejoined_string = ' '.join(split) 
    # rejoined_string = ' '.join(no_punctuation_no_stopwords) 
    #print(rejoined_string)
    return rejoined_string
             
    #word frequency before stopwords taken out
    word_freq = {}
    words = []
    for word in split:
        if word in words:
           word_freq[word] += 1
        else:
            words.append(word)
            word_freq[word] = 1
    #order by values
    sort_freqs = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
    #print the top 10 most used words
    for i in sort_freqs[:10]:
        print(i[0], i[1])
            
    #word frequency after stopwords taken out
    word_freq = {}
    words = []
    for word in no_punctuation_no_stopwords:
        if word in words:
           word_freq[word] += 1
        else:
            words.append(word)
            word_freq[word] = 1
    #order by values
    sort_freqs = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
    #print the top 10 most used words
    for i in sort_freqs[:10]:
        print(i[0], i[1])

In [4]:
##################################################
############# for sentiment analysis #############
##################################################

rejoined_string = preprocessingForSentimentAnalysis(all_the_scripts)

##tokenize by sentence
def tokenizeBySentence(string):
    from nltk.tokenize import sent_tokenize
    tokenized_script = sent_tokenize(string) 
    # tokenized_script = sent_tokenize(rejoined_string)
    #print(tokenized_script)
    return tokenized_script

#this is now the whole script tokenized by each sentence
tokenized_script = tokenizeBySentence(all_the_scripts)

In [6]:
###split script by character name
def splitCharacterByName(script):
    
    script_dict = {}
    character_names = []
    current_character = ''
    split_script = []

    for line in script:
        line = line.split(":")
        if line not in split_script:
            # print(line)
            split_script.append(line)
    for line in split_script:
        # if len(line)==2:
        #     print(line[0])         
        #if line contains character heading and character not in list of character names
        if len(line)==2 and line[0] not in character_names and len(line[0])<20: #and line[0].isupper()
            # print(line[1])
            # print(line[0])
            character_names.append(line[0])            
            current_character = line[0]    
            script_dict[current_character] = line[1]          
            # for element in line[1]:
            #     if punctuation not in element or stopwords not in element:
            #         script_dict_no_punct_no_stopwords[line[0]] = line[1].lower() #make the words lowercase
        #if line contains character heading and character IS in list of character names
        elif len(line)==2 and line[0] in character_names: #len(line)==2 and 
            current_character = line[0]    
            script_dict[current_character] += line[1] 
            # if len(line)==1:
            #     script_dict[current_character] += line
            # print(line)
                # if punctuation not in line or stopwords not in line:
                #     script_dict_no_punct_no_stopwords[current_character] += line.lower() #make the words lowercase
        #if line does not contain character heading, add it to the most recent character
        elif len(line)==1: 
            if current_character == '':
                pass
            else:    
                print(current_character)
                print(line[0])
                script_dict[current_character] += line[0] 
    print(script_dict)
    return script_dict

#this is a dictionary containing each character heading (like "Morty") and every
#line that comes after that unique character heading
script_dict = splitCharacterByName(tokenized_script)
# len(script_dict['Rick'])

Rick
Next time I need a species sterilized, who's gonna do it for me?
Rick
You?
Rick
I-Is that the Vindibeacon?
Rick
We're being called to assemble by the Vindicators!
Rick
Let it go to voicemail!
Morty
They're first line of defense against evil!
Morty
They're the guardians of the unguarded!
Rick
They're a bunch of drama queens that spend an hour talking and twenty minutes jumping around while shit blows up.
Rick
They're a phase.
Rick
We did one, it was the big event of that summer.
Rick
Let it die.
Morty
[holds up a "Morty Adventure Card" with ten squares and all but one containing a Morty-shaped stamp mark] Read 'em and weep.
Rick
[takes the card] Fine!
Rick
But don't say I didn't warn you.
Rick
[presses a button on the Vindibeacon and speaks into it] Vindicator command ship, beacon received!
Rick
We're doing Vindicators Twoooooooooo!
Rick
Uh, Morty, you might want to freeze some sperm.
Supernova
Once in range, Million Ants will scan for pheromone activity, which should lead us to th

In [7]:
##tokenize by word
def tokenizeByWord(script):
    from nltk.tokenize import word_tokenize
    # tokenized_script = sent_tokenize(all_the_scripts) 
    # check_list = []
    for character in script:
        script[character] = word_tokenize(script[character])
    
    return script

script_dict = tokenizeByWord(script_dict)
# len(script_dict['Rick'])

In [10]:
def mainCharacters(wordDict):

    main_characters = []
    ##determine main characters
    #order characters by number of words they said
    for character in wordDict:
        if len(wordDict[character]) > 175:
            main_characters.append(character)
            # print(character)    
        # print(sorted(wordDict.values()))#print(character, len(wordDict[character]))#word.lower()
    # main_characters
    
    # len(wordDict['Rick'])
    for character in main_characters:
        if 'Rick' in character and character!='Rick':
            wordDict['Rick'] += wordDict[character]
    for character in main_characters:
        if 'Morty' in character and character!='Morty':
            wordDict['Morty'] += wordDict[character]
    # len(wordDict['Rick'])
    
    main_characters = []
    for character in wordDict:
        if len(wordDict[character]) > 175:
            main_characters.append(character)

    return main_characters

main_characters = mainCharacters(script_dict)
# len(script_dict['Rick'])

character_script = splitCharacterByName(tokenized_script)

Rick
Next time I need a species sterilized, who's gonna do it for me?
Rick
You?
Rick
I-Is that the Vindibeacon?
Rick
We're being called to assemble by the Vindicators!
Rick
Let it go to voicemail!
Morty
They're first line of defense against evil!
Morty
They're the guardians of the unguarded!
Rick
They're a bunch of drama queens that spend an hour talking and twenty minutes jumping around while shit blows up.
Rick
They're a phase.
Rick
We did one, it was the big event of that summer.
Rick
Let it die.
Morty
[holds up a "Morty Adventure Card" with ten squares and all but one containing a Morty-shaped stamp mark] Read 'em and weep.
Rick
[takes the card] Fine!
Rick
But don't say I didn't warn you.
Rick
[presses a button on the Vindibeacon and speaks into it] Vindicator command ship, beacon received!
Rick
We're doing Vindicators Twoooooooooo!
Rick
Uh, Morty, you might want to freeze some sperm.
Supernova
Once in range, Million Ants will scan for pheromone activity, which should lead us to th

In [11]:
#redirecting
import sys
orig_stdout = sys.stdout
#redirecting

morty_lines = ''
rick_lines = ''
for character in character_script:
    # print(type(character_script[character]))
    if character == 'Morty':
        f = open('morty_lines.txt', 'w') #redirecting
        sys.stdout = f
        print(character_script[character]) #redirecting
        morty_lines += character_script[character]
    elif character == 'Rick':
        f = open('rick_lines.txt', 'w') #redirecting
        sys.stdout = f
        print(character_script[character]) #redirecting
        rick_lines += character_script[character]
#redirecting
sys.stdout = orig_stdout
f.close()

In [12]:
#################################################
############## Markov Chains ####################
#################################################
from collections import defaultdict
# from Text_Analysis_Functions import RickLines, MortyLines
# import nltk
# from nltk.tokenize import word_tokenize

def MC(a): #MC = Markov Chain
    words = a #Tokenized words
    m_dict = defaultdict(list)
    for current_word, next_word in zip(words[0:-1], words[1:]):
        m_dict[current_word].append(next_word)
    m_dict = dict(m_dict)
    return m_dict


split_morty_lines = morty_lines.split(' ')
mc_guess_morty = MC(split_morty_lines)
#print(mc_guess_morty)

split_rick_lines = rick_lines.split(' ')
mc_guess_rick = MC(split_rick_lines)

In [15]:
import random
def generate_sent(chain, count=25):
    word1 = random.choice(list(chain.keys()))
    sentence = word1.capitalize()
    for i in range(count-1):
        word2 = random.choice(chain[word1])
        word1 = word2
        sentence += ' ' + word2
    sentence += '.'
    return sentence
generate_sent(mc_guess_morty)

"Budgets and all, could you the baskets.I'll disarm the drunkenly-improvised neutrino bomb.There's a way that I don't like us. Yeah, it in there. I... think."

In [17]:
sentence1 = generate_sent(mc_guess_morty)
sentence1

'Realization] Oh, I think... no point!You just proving my right you are, nobody wants our last moment alive to be spent knowing how few fucks.'

In [22]:
sentence2 = generate_sent(mc_guess_morty)
sentence2

"Up. [gagging] Greater good?! Never said it's the platform] I'll cover that bet.I get it.I-I'm so the universe is an asshole, it yourself. Just focus."

In [23]:
sentence3 = generate_sent(mc_guess_morty)
sentence3

"Out to? Right? But.. she was trying to be pretty sure we don't need jackets.[takes his jacket off] And I'm pretty sure we put it."

In [24]:
sentence4 = generate_sent(mc_guess_morty)
sentence4

'Blackout drunk. In a superhero. Rick and "bad" are artificial constructs. [annoyed] Get it out. That\'s not the universe itself is an even bigger threat.'